# Convolutional Neural Network

### Importing the libraries

In [1]:
%pip install --upgrade tensorflow keras Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2022-03-23 20:08:36.982136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 20:08:36.982152: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [29]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [30]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='leaky_relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [31]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [32]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='leaky_relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [33]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [34]:
cnn.add(tf.keras.layers.Dense(units=128, activation='leaky_relu'))
cnn.add(tf.keras.layers.Dense(units=64, activation='leaky_relu'))


### Step 5 - Output Layer

In [35]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [36]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [38]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
250/250 [==============================] - 559s 2s/step - loss: 0.6342 - accuracy: 0.6366 - val_loss: 0.5913 - val_accuracy: 0.6930
Epoch 2/20
250/250 [==============================] - 37s 149ms/step - loss: 0.5715 - accuracy: 0.6964 - val_loss: 0.5759 - val_accuracy: 0.7035
Epoch 3/20
250/250 [==============================] - 38s 153ms/step - loss: 0.5264 - accuracy: 0.7311 - val_loss: 0.5251 - val_accuracy: 0.7400
Epoch 4/20
250/250 [==============================] - 39s 154ms/step - loss: 0.4998 - accuracy: 0.7561 - val_loss: 0.5007 - val_accuracy: 0.7640
Epoch 5/20
250/250 [==============================] - 38s 153ms/step - loss: 0.4652 - accuracy: 0.7800 - val_loss: 0.5199 - val_accuracy: 0.7535
Epoch 6/20
250/250 [==============================] - 39s 155ms/step - loss: 0.4491 - accuracy: 0.7886 - val_loss: 0.4877 - val_accuracy: 0.7770
Epoch 7/20
250/250 [==============================] - 39s 154ms/step - loss: 0.4369 - accuracy: 0.7958 - val_loss: 0.4764 - val_accu

## Part 4 - Making a single prediction

In [41]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [42]:
print(prediction)

dog


## Fim